# MEEGnobis: basic usage

MEEGnobis can be used to perform multivariate analyses on M/EEG datasets over time. It interfaces with [MNE-Python](https://www.martinos.org/mne/stable/index.html) and assumes that all the data have been already preprocessed according to your taste.

The main function is `compute_temporal_rdm`, which we import in the following cell

In [1]:
import numpy as np
from meegnobis.rsa import compute_temporal_rdm, make_pseudotrials
from meegnobis.testing import generate_epoch
from meegnobis.log import log, logging
# increase log level for this notebook
log.setLevel(logging.ERROR)

Please take time to read the docstring of the function

In [2]:
print(compute_temporal_rdm.__doc__)

Computes pairwise metric across time

    Arguments
    ---------
    epoch : instance of mne.Epoch
    targets : array (n_trials,)
        target (condition) for each trials
    metric: str | BaseMetric | sklearn estimator
        type of metric to use, one of 'cityblock', 'correlation', 'cosine',
        'dice', 'euclidean', 'hamming', 'sqeuclidean' for distances.
        Alternatively, any object with attributes fit/score, similar to
        sklearn estimators.
    cv : instance of sklearn cross-validator
        Cross-validator used to split the data into training/test datasets
        (default StratifiedShuffleSplit(n_splits=10, test_size=0.5)
    cv_normalize_noise : str | None (default None)
        Multivariately normalize the trials before applying the metric
        (distance or classification). Normalization is performed with
        cross-validation, that is the covariance matrix is estimated in the
        training set, and then applied to the test set. Normalization is
  

In order to run temporal RDM we need 

- An `mne.Epoch` object containing the data
- A `targets` array containing the target/class information for each trial
- A `metric` to be used on the data. We can pass any object that has scikit-learn's `fit/predict` constructs, such as any scikit-learn's classifier. Alternatively, one can pass a string to use a specific distance metric.
- A cross-validation object to perform, well, cross-validation. By default we use `StratifiedShuffleSplit` from scikit-learn, with 10 splits and `test_size = 0.5`. This will split the data randomly for 10 times to generate training/testing sets of equal size.

Let's see an example with made up data:

In [3]:
epoch = generate_epoch(n_epochs_cond=10,
                       n_conditions=5, 
                       n_channels=10,
                       sfreq=200.0, 
                       n_times=100)

50 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped


In [4]:
print(epoch)

<EpochsArray  |  n_events : 50 (all good), tmin : 0.0 (s), tmax : 0.495 (s), baseline : None, ~417 kB, data loaded,
 '0': 10, '1': 10, '2': 10, '3': 10, '4': 10>


So we generated 50 trials with 10 channels/sensors each, with 5 conditions total (thus 10 trials for each condition), and time between 0 and 0.495 s. We're going to use this dataset for different example analyses. First, we need to baseline correct our data (in this case, we do it by considering as baseline the first 100ms:

In [5]:
epoch.apply_baseline((0, 0.1))

Applying baseline correction (mode: mean)


<EpochsArray  |  n_events : 50 (all good), tmin : 0.0 (s), tmax : 0.495 (s), baseline : (0, 0.1), ~417 kB, data loaded,
 '0': 10, '1': 10, '2': 10, '3': 10, '4': 10>

Then, to make things more interesting, we can just rename the conditions with semanticly meaningful labels

In [6]:
labels = ['face', 'shoe', 'house', 'cat', 'dog']
epoch.event_id = {lbl: idx for idx, lbl in enumerate(labels)}
print(epoch)

<EpochsArray  |  n_events : 50 (all good), tmin : 0.0 (s), tmax : 0.495 (s), baseline : (0, 0.1), ~417 kB, data loaded,
 'cat': 10, 'dog': 10, 'face': 10, 'house': 10, 'shoe': 10>


## Example 1: cross-validated mahalanobis distance (crossnobis)

We will run a form of RSA that is noise-normalized using the sensor-wise covariance estimates across the entire epoch using only the trials in the training set. We are going to use the Euclidean distance as a metric, since the Euclidean distance on such normalized data is in fact the Mahalanobis distance. We are going to use only one split in the training/test set for speed, but you might want to use more splits with real data.

In [7]:
from sklearn.model_selection import StratifiedShuffleSplit
cv = StratifiedShuffleSplit(n_splits=1, test_size=0.5)

In [8]:
# we are going to take as targets the event labels
targets = [labels[e] for e in epoch.events[:, -1]]
print(targets)

['face', 'face', 'face', 'face', 'face', 'face', 'face', 'face', 'face', 'face', 'shoe', 'shoe', 'shoe', 'shoe', 'shoe', 'shoe', 'shoe', 'shoe', 'shoe', 'shoe', 'house', 'house', 'house', 'house', 'house', 'house', 'house', 'house', 'house', 'house', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog']


Because we're running a distance metric, we need to set the argument `mean_groups` to `True`: this will average the trials for each class or group in both the training/test set and then compare the average patterns between sets.

In [9]:
rdm, target_pairs = compute_temporal_rdm(
    epoch, targets, cv=cv, metric='euclidean', cv_normalize_noise='epoch', mean_groups=True)

50 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Estimating covariance using SHRUNK
Done.
Using cross-validation to select the best estimator.
Number of samples used : 2500
[done]
log-likelihood on unseen data (descending order):
   shrunk: -359.709
selecting best estimator: shrunk
estimated rank (mag): 10
Setting small MEG eigenvalues to zero.
Not doing PCA for MEG.


In [10]:
rdm.shape, len(target_pairs)

((15, 5050), 15)

As a result we got an array of shape `(15, 5050)`. Each row correspond to a pair of categories (labeled in `target_pairs`), whereas each column correspond to a pair of timepoints on which the RDM is computed on; because we used a symmetric metric, only 5050 (instead of 100 * 100) pairs of timepoints are returned, corresponding to the upper triangular matrix (diagonal included) over time.

In [11]:
print(target_pairs)

['cat+cat', 'cat+dog', 'cat+face', 'cat+house', 'cat+shoe', 'dog+dog', 'dog+face', 'dog+house', 'dog+shoe', 'face+face', 'face+house', 'face+shoe', 'house+house', 'house+shoe', 'shoe+shoe']


## Example 2: decoding

We can use the same code to run decoding using any of scikit-learn's classifiers. We're going to use an SVM with linear kernel, without noise-normalization, and increasing the number of splits to 4 this time. Because we're using a classifier, we do not want to average the trials within each class, so we set `mean_groups` to `False`. The function will perform all pairwise binary classifications.

In [12]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
cv = StratifiedShuffleSplit(n_splits=4, test_size=0.5)

In [13]:
rdm, target_pairs = compute_temporal_rdm(epoch, targets, cv=cv, metric=clf)

50 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
50 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
50 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
50 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped


In [14]:
rdm.shape, len(target_pairs)

((15, 10000), 15)

Now the result is a `(15, 10000)` array. Because the classifier is not symmetric over time (e.g., training at time `t0` and testing at time `t1` is different than training at time `t1` and testing at time `t0`), the entire matrix is returned. The order is rows first, then columns, so that one can create a tensor `n_pairwise_targets X n_time X n_time` by simply using reshape:

In [15]:
rdm = rdm.reshape((-1, 100, 100))
rdm.shape

(15, 100, 100)

## Example 3: decoding with "pseudotrials"

SNR can be increased by using "pseudotrials", generated by randomly averaging subsets of trials. An approach that is used in the literature (see e.g., Cichy et al., 2014) is to repeat this averaging 100 times, randomly selecting trials to average every time, and testing the classifiers using a leave-one-sample-out scheme. We can easily do this in `meegnobis` using `make_pseudotrials`:

In [16]:
print(make_pseudotrials.__doc__)

Create pseudotrials by averaging within each group defined in `targets`
    a number `navg` of trials. The trials are randomly divided into groups of
    size `navg` for each target. If the number of trials in a group is not
    divisible by `navg`, one pseudotrials will be created by averaging the
    remainder trials.

    Parameters
    ----------
    epoch : mne.Epoch
    targets : (n_epochs,)
    navg : int
        number of trials to average
    rng : random number generator

    Returns
    -------
    avg_epoch : mne.Epoch
        epoch with ceil(n_epochs/navg) trials
    avg_targets : (ceil(n_epochs/navg),)
        unique targets corresponding to the avg_trials
    


In our example dataset we have 10 trials for each condition, so we can average three trials at a time to generate four samples for each condition:

In [17]:
avg_epoch, avg_targets = make_pseudotrials(epoch, targets, navg=3)

20 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped


In [18]:
print(avg_epoch)

<EpochsArray  |  n_events : 20 (all good), tmin : 0.495 (s), tmax : 0.99 (s), baseline : (0, 0.1), ~182 kB, data loaded,
 'cat': 4, 'dog': 4, 'face': 4, 'house': 4, 'shoe': 4>


In [19]:
print(avg_targets)

['cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'face', 'face', 'face', 'face', 'house', 'house', 'house', 'house', 'shoe', 'shoe', 'shoe', 'shoe']


We can  repeat this 5 times, and perform classification every time using a leave-one-out-sample cross-validation scheme, and then averaging across the 5 folds. We're still going to use `StratifiedShuffleSplit`, but this time we ask a test set with size exactly equal to the number of unique targets. In this way we'll have a single sample for every category in the test set.

We can write the code easily as follows:

In [20]:
n_unique_targets = len(np.unique(targets))
clf = SVC(kernel='linear')

out = []
for _ in range(5):
    # we reset the cv inside the loop so that it gets random splits every time
    cv = StratifiedShuffleSplit(n_splits=1, test_size=n_unique_targets)    
    # get randomly generate pseudotrials
    avg_epoch, avg_targets = make_pseudotrials(epoch, targets, navg=3)
    # compute RDM over time
    rdm, target_pairs = compute_temporal_rdm(avg_epoch, avg_targets, cv=cv, metric=clf)
    out.append(rdm)

20 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
20 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
20 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
20 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
20 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
20 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
20 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
20 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
20 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
20 matching events found
No baseline correctio

In [21]:
len(out), out[0].shape

(5, (15, 10000))

And we can then finally average to obtain the final RDM

In [22]:
final_rdm = np.mean(out, axis=0)
final_rdm.shape

(15, 10000)